## Dependencies

In [1]:
# System dependencies
from os import listdir
import time as clock
from datetime import timedelta
from IPython.display import clear_output
import pickle

In [2]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# Custom data handling methods
import prep_IO

In [4]:
# Directory of pre-determined lustre failed jobs
#1) emailed set, 2) emailed set
#first_src = './lustre_fail_set.txt'

# 3) 9/21 - 9/23 4) 10/14
#rem_src = [ '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2020-09-21.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2020-09-22.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2020-09-23.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2020-10-14.txt']

#my_list = prep_IO.acct_file_to_searchset( acct_chunks=False, partial_txt=first_src, full_txt=False )

#for item in rem_src:
#    my_list += prep_IO.acct_file_to_searchset( full_txt=item )

#len(my_list)

In [5]:
src =  './lustre_set.pkl'

In [6]:
# Given: 'JobID|User|Account|Start|End|Submit|Partition|Timelimit|JobName|State|NNodes|ReqCPUS|NodeList\n'
# DNI: User, Account, JobName
# Else: OK
f = open(src,"rb")
test_out = pickle.load( f )
len(test_out)

84243

## Data

In [19]:
save_list = []

In [18]:
sample = test_out[500:1000]

In [20]:
search_out_B = prep_IO.deep_search_host( sample, jobids=True )
save_list += search_out_B

print(len( search_out_B ))
print(len( save_list ))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 579: invalid start byte

In [17]:
save_thing = save_list
f = open("focus_lustre/lustre_set_1000.pkl","wb")
pickle.dump( save_thing,f )
f.close()

#### PAUSE FOR DEBUG

In [ ]:
def fill_host_source( deep_out_dict ):
    return { search_key : host_to_info_dict( source_dict["Source"][0] )
                for search_key,source_dict in deep_out_dict.items() }

def host_to_info_dict( zip_txt, find_id=False ):
    l = zip_txt.find("comet")
    r = zip_txt.rfind("comet")
    
    if l != r:
        return zip_txt
    
    contents = prep_IO.unzip_txt( zip_txt )
    #host_name = contents[1].partition(" ")[2][:11]
    info_dict = { "Schemas":{}, "Specs":[],
                  "Data":{}, "Jobid(s)":[]
                }
    data_start = 0
    
    for i in range(len( contents )):
        line = contents[ i ]
        
        # Spec line
        if line[0] == "$":
            info_dict["Specs"].append( prep_IO.format_spec( line ) )
        
        # Schema line    
        elif line[0] == "!":
            info_dict["Schemas"].update( prep_IO.format_schema( line ) )
                
        # Start of variable content options
        else:
            data_start = i
    
    curr_header = ''
    for j in range( data_start, len(contents) ):
        line = contents[ j ]

        # Header line
        if prep_IO.check_header( line ):
            header_dict = prep_IO.format_header( line )
            curr_header = line
                                        
            if header_dict:
                info_dict["Data"][ curr_header ] = {} 
                    
            if prep_IO.check_job( header_dict["Jobid"] ):
                curr_jobid = header_dict["Jobid"]
                info_dict["Jobid(s)"].append( curr_jobid )

        # Data line
        else:
            curr_data = prep_IO.format_data( line )
            info_dict["Data"][curr_header].update( curr_data )
    
    return info_dict

In [ ]:
deep_out = deep_search_host( test_out, jobids=True )
deep_out

In [ ]:
test_key = ('comet-14-72',
  '2020-05-13T02:11:38',
  '2020-05-13T05:07:34',
  '33301074')

host_to_info_dict( deep_out[test_key]["Source"][0] )

#### RESUME

In [ ]:
keys = list(search_out.keys())
len(keys)

In [ ]:
sample_key = keys[0]
sample = search_out[ sample_key ]
print( sample_key )
print( sample.keys() )

In [ ]:
sample["Source Files"]

In [ ]:
sample['Acct Info']

In [ ]:
sample['Host Info'].keys()

In [ ]:
sample['Host Info']["Specs"]

In [ ]:
sample['Host Info']["Timely Data"]

# Save search_out for demo_ use

In [ ]:
save_thing = my_list
f = open("lustre_set.pkl","wb")
pickle.dump( save_thing,f )
f.close()